# 🧠 MARVIS Getting Started Guide

Welcome to **MARVIS** (Modality Adaptive Reasoning over VISualizations)! This notebook will guide you through using MARVIS to make predictions on **tabular**, **image**, and **audio** data using the unified `MarvisTsneClassifier` interface.

## What is MARVIS?

MARVIS is a novel approach that:
1. **Embeds** your data into high-dimensional representations
2. **Visualizes** the data using t-SNE or other dimensionality reduction techniques
3. **Reasons** about the visualization using Vision Language Models (VLMs)
4. **Predicts** class labels based on visual patterns

Let's start by importing the necessary libraries and setting up our environment.

In [ ]:
import sys
import os
import numpy as np
import matplotlib.pyplot as plt
import warnings
from pathlib import Path

# Add MARVIS to Python path
marvis_root = Path.cwd().parent
sys.path.insert(0, str(marvis_root))

# Import MARVIS modules
from marvis.models.marvis_tsne import MarvisTsneClassifier
from marvis.utils.audio_utils import create_synthetic_audio
from sklearn.datasets import make_classification, load_digits
from sklearn.model_selection import train_test_split
from IPython.display import display, Image as IPImage, Audio
import tempfile
import shutil

# Suppress warnings for cleaner output
warnings.filterwarnings('ignore')

print("🎉 MARVIS libraries imported successfully!")
print(f"📁 MARVIS root directory: {marvis_root}")

## 📊 Part 1: Tabular Data Classification

Let's start with tabular data - the most common use case. We'll create a synthetic dataset and show how MARVIS can classify it using visual reasoning.

In [ ]:
# Create synthetic tabular dataset
print("📊 Creating synthetic tabular dataset...")

X_tabular, y_tabular = make_classification(
    n_samples=200,
    n_features=10,
    n_classes=3,
    n_informative=8,
    n_redundant=1,
    n_clusters_per_class=1,
    class_sep=1.2,
    random_state=42
)

# Split into train/test
X_train_tab, X_test_tab, y_train_tab, y_test_tab = train_test_split(
    X_tabular, y_tabular, test_size=0.3, random_state=42, stratify=y_tabular
)

class_names_tab = ["Class A", "Class B", "Class C"]

print(f"✅ Dataset created:")
print(f"   • Training samples: {len(X_train_tab)}")
print(f"   • Test samples: {len(X_test_tab)}")
print(f"   • Features: {X_tabular.shape[1]}")
print(f"   • Classes: {len(class_names_tab)}")
print(f"   • Class distribution: {np.bincount(y_tabular)}")

In [ ]:
# Initialize MARVIS classifier for tabular data
print("🔧 Initializing MARVIS t-SNE classifier for tabular data...")

tabular_classifier = MarvisTsneClassifier(
    modality="tabular",
    vlm_model_id="Qwen/Qwen2.5-VL-3B-Instruct",  # Fast and efficient VLM
    tsne_perplexity=15,
    tsne_max_iter=500,
    use_3d=False,
    use_knn_connections=True,
    nn_k=5,
    max_vlm_image_size=800,
    image_dpi=100,
    use_semantic_names=True,
    seed=42
)

print("✅ Tabular classifier initialized!")

In [ ]:
# Fit the classifier
print("🏋️ Training MARVIS on tabular data...")

tabular_classifier.fit(
    X_train_tab, 
    y_train_tab, 
    X_test_tab, 
    class_names=class_names_tab,
    task_type='classification'  # Explicitly specify task type for synthetic data
)

print("✅ Tabular classifier trained successfully!")

In [ ]:
# Make predictions and evaluate
print("🔮 Making predictions on tabular test data...")

# Create temporary directory for outputs
temp_dir = tempfile.mkdtemp(prefix='marvis_demo_')
print(f"📁 Saving outputs to: {temp_dir}")

try:
    # Evaluate with detailed results and save outputs
    tabular_results = tabular_classifier.predict_with_detailed_results(
        X_test_tab[:10],  # Use first 10 test samples for demo
        y_test_tab[:10],
        save_visualizations=True,
        output_dir=temp_dir
    )
    
    print("\n📈 Tabular Classification Results:")
    accuracy = tabular_results.get('accuracy', 'N/A')
    total_samples = len(X_test_tab[:10])
    completed_samples = tabular_results.get('completed_samples', total_samples)
    
    print(f"   • Accuracy: {accuracy}")
    print(f"   • Completed samples: {completed_samples}")
    
    # Display some predictions
    predictions = tabular_results.get('predictions', [])
    if predictions:
        print("\n🎯 Sample Predictions:")
        for i, pred in enumerate(predictions[:3]):
            predicted_class = pred if isinstance(pred, str) else class_names_tab[pred] if pred < len(class_names_tab) else f"Class {pred}"
            true_class = class_names_tab[y_test_tab[i]]
            print(f"   Sample {i+1}: Predicted='{predicted_class}', True='{true_class}'")
    
except Exception as e:
    print(f"❌ Error during evaluation: {e}")
    print("This might be due to VLM API limitations. The classifier is still trained successfully.")

In [ ]:
# Display the visualization
print("🖼️ Displaying t-SNE visualization for tabular data...")

# Look for visualization files in the temp directory
viz_files = list(Path(temp_dir).glob("*.png"))

if viz_files:
    # Display the most recent visualization
    latest_viz = max(viz_files, key=lambda p: p.stat().st_mtime)
    print(f"📊 Found visualization: {latest_viz.name}")
    
    # Display using IPython
    display(IPImage(filename=str(latest_viz), width=600))
    
    print("\n🎨 Visualization Features:")
    print("   • Colored points represent training data by class")
    print("   • Gray squares represent test data")
    print("   • KNN connections show nearest neighbors")
    print("   • The VLM analyzes this visualization to make predictions")
else:
    print("⚠️ No visualization files found. This might be due to VLM API limitations.")
    print("The visualization would normally show:")
    print("   • t-SNE projection of high-dimensional tabular data")
    print("   • Color-coded classes for visual pattern recognition")
    print("   • KNN connections between similar data points")

## 🖼️ Part 2: Image Data Classification

Now let's try MARVIS with image data. We'll use the digits dataset as an example of computer vision tasks.

In [ ]:
# Load digits dataset (images)
print("🖼️ Loading digits dataset (image data)...")

digits = load_digits()
X_images = digits.data  # 8x8 pixel images flattened to 64 features
y_images = digits.target

# Use only first 3 classes for simplicity
mask = y_images < 3
X_images = X_images[mask]
y_images = y_images[mask]

# Split into train/test
X_train_img, X_test_img, y_train_img, y_test_img = train_test_split(
    X_images, y_images, test_size=0.3, random_state=42, stratify=y_images
)

class_names_img = ["Digit 0", "Digit 1", "Digit 2"]

print(f"✅ Digits dataset loaded:")
print(f"   • Training samples: {len(X_train_img)}")
print(f"   • Test samples: {len(X_test_img)}")
print(f"   • Image size: 8x8 pixels (64 features)")
print(f"   • Classes: {class_names_img}")

# Visualize some sample digits
fig, axes = plt.subplots(1, 6, figsize=(12, 2))
for i in range(6):
    axes[i].imshow(X_train_img[i].reshape(8, 8), cmap='gray')
    axes[i].set_title(f"Class: {class_names_img[y_train_img[i]]}")
    axes[i].axis('off')
plt.suptitle("Sample Digit Images")
plt.tight_layout()
plt.show()

In [ ]:
# Initialize MARVIS classifier for image data
print("🔧 Initializing MARVIS t-SNE classifier for image data...")

image_classifier = MarvisTsneClassifier(
    modality="vision",  # Specify vision modality for image data
    vlm_model_id="Qwen/Qwen2.5-VL-3B-Instruct",
    tsne_perplexity=20,
    tsne_max_iter=500,
    use_3d=False,
    use_knn_connections=True,
    nn_k=5,
    max_vlm_image_size=800,
    use_semantic_names=True,
    seed=42
)

print("✅ Image classifier initialized!")

In [ ]:
# Fit the image classifier
print("🏋️ Training MARVIS on image data...")

image_classifier.fit(
    X_train_img,
    y_train_img,
    X_test_img[:20],  # Use subset for demo
    class_names=class_names_img,
    task_type='classification'  # Explicitly specify task type
)

print("✅ Image classifier trained successfully!")

In [ ]:
# Make predictions on image data
print("🔮 Making predictions on image test data...")

# Create new temp directory for image outputs
temp_dir_img = tempfile.mkdtemp(prefix='marvis_image_demo_')
print(f"📁 Saving image outputs to: {temp_dir_img}")

try:
    # Evaluate on a small subset
    image_results = image_classifier.predict_with_detailed_results(
        X_test_img[:8],  # Use first 8 test samples
        y_test_img[:8],
        save_visualizations=True,
        output_dir=temp_dir_img
    )
    
    print("\n📈 Image Classification Results:")
    accuracy = image_results.get('accuracy', 'N/A')
    completed_samples = image_results.get('completed_samples', len(X_test_img[:8]))
    
    print(f"   • Accuracy: {accuracy}")
    print(f"   • Completed samples: {completed_samples}")
    
    # Display some predictions
    predictions = image_results.get('predictions', [])
    if predictions:
        print("\n🎯 Sample Image Predictions:")
        for i, pred in enumerate(predictions[:3]):
            predicted_class = pred if isinstance(pred, str) else class_names_img[pred] if pred < len(class_names_img) else f"Class {pred}"
            true_class = class_names_img[y_test_img[i]]
            print(f"   Sample {i+1}: Predicted='{predicted_class}', True='{true_class}'")
                
except Exception as e:
    print(f"❌ Error during image evaluation: {e}")
    print("The classifier is trained, but prediction may require VLM API access.")

In [ ]:
# Display the image visualization
print("🖼️ Displaying t-SNE visualization for image data...")

# Look for visualization files
viz_files_img = list(Path(temp_dir_img).glob("*.png"))

if viz_files_img:
    latest_viz_img = max(viz_files_img, key=lambda p: p.stat().st_mtime)
    print(f"📊 Found image visualization: {latest_viz_img.name}")
    
    display(IPImage(filename=str(latest_viz_img), width=600))
    
    print("\n🎨 Image Visualization Features:")
    print("   • Each point represents a digit image in 2D t-SNE space")
    print("   • Colors distinguish different digit classes (0, 1, 2)")
    print("   • Similar digits cluster together in the visualization")
    print("   • VLM identifies patterns to classify new digits")
else:
    print("⚠️ No image visualization found. This would show:")
    print("   • t-SNE embedding of 64-dimensional digit features")
    print("   • Clear clustering of similar digit patterns")
    print("   • Visual separation between different digit classes")

## 🎵 Part 3: Audio Data Classification

Finally, let's explore MARVIS's capabilities with audio data. We'll create synthetic audio samples with different frequencies to demonstrate audio classification.

In [ ]:
# Create synthetic audio dataset
print("🎵 Creating synthetic audio dataset...")

# Create temporary directory for audio files
audio_temp_dir = tempfile.mkdtemp(prefix='clam_audio_')
print(f"📁 Creating audio files in: {audio_temp_dir}")

# Generate synthetic audio samples
audio_paths = []
audio_labels = []
class_names_audio = ["Low Tone", "Mid Tone", "High Tone"]

# Create samples for each class
samples_per_class = 15
duration = 2.0  # 2 seconds
sample_rate = 16000

try:
    import soundfile as sf
    
    for class_idx, class_name in enumerate(class_names_audio):
        base_freq = 200 + class_idx * 300  # 200Hz, 500Hz, 800Hz
        
        for sample_idx in range(samples_per_class):
            # Add some variation to the frequency
            freq_variation = np.random.uniform(-50, 50)
            frequency = base_freq + freq_variation
            
            # Create synthetic audio
            audio = create_synthetic_audio(
                frequency=frequency,
                duration=duration,
                sample_rate=sample_rate,
                noise_level=0.1
            )
            
            # Save to file
            audio_path = Path(audio_temp_dir) / f"{class_name.lower().replace(' ', '_')}_{sample_idx:02d}.wav"
            sf.write(str(audio_path), audio, sample_rate)
            
            audio_paths.append(str(audio_path))
            audio_labels.append(class_idx)
    
    # Convert to numpy arrays
    audio_labels = np.array(audio_labels)
    
    # Split into train/test
    audio_train_paths, audio_test_paths, y_train_audio, y_test_audio = train_test_split(
        audio_paths, audio_labels, test_size=0.3, random_state=42, stratify=audio_labels
    )
    
    print(f"✅ Audio dataset created:")
    print(f"   • Training samples: {len(audio_train_paths)}")
    print(f"   • Test samples: {len(audio_test_paths)}")
    print(f"   • Duration: {duration}s per sample")
    print(f"   • Classes: {class_names_audio}")
    print(f"   • Frequencies: ~200Hz, ~500Hz, ~800Hz")
    
    # Display audio waveforms
    fig, axes = plt.subplots(3, 1, figsize=(12, 8))
    
    for class_idx in range(3):
        # Load and plot first sample of each class
        sample_path = [p for p, l in zip(audio_train_paths, y_train_audio) if l == class_idx][0]
        audio_data, sr = sf.read(sample_path)
        
        time_axis = np.arange(len(audio_data)) / sr
        axes[class_idx].plot(time_axis[:1000], audio_data[:1000])  # Plot first 1000 samples
        axes[class_idx].set_title(f"{class_names_audio[class_idx]} - Sample Waveform")
        axes[class_idx].set_xlabel("Time (s)")
        axes[class_idx].set_ylabel("Amplitude")
        axes[class_idx].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # Play a sample audio (first test sample)
    if audio_test_paths:
        sample_audio, sample_sr = sf.read(audio_test_paths[0])
        print(f"\n🎧 Playing sample audio ({class_names_audio[y_test_audio[0]]})...")
        display(Audio(sample_audio, rate=sample_sr))

except ImportError:
    print("⚠️ soundfile not available. Skipping audio file creation.")
    print("Audio classification would work with real audio files.")
    audio_available = False
except Exception as e:
    print(f"⚠️ Error creating audio files: {e}")
    audio_available = False
else:
    audio_available = True

In [ ]:
if audio_available:
    # Initialize MARVIS classifier for audio data
    print("🔧 Initializing MARVIS audio classifier...")
    
    try:
        audio_classifier = MarvisTsneClassifier(
            modality="audio",
            whisper_model="tiny",  # Fast Whisper model for embeddings
            vlm_model_id="Qwen/Qwen2.5-VL-3B-Instruct",
            tsne_perplexity=10,
            tsne_max_iter=300,
            use_3d=False,
            use_knn_connections=True,
            nn_k=3,
            max_vlm_image_size=800,
            device="auto",  # Use auto device detection
            seed=42
        )
        
        print("✅ Audio classifier initialized!")
        
    except ImportError as e:
        print(f"⚠️ Audio classifier not available: {e}")
        print("This requires additional audio dependencies.")
        audio_classifier = None
else:
    audio_classifier = None

In [ ]:
if audio_available and audio_classifier is not None:
    # Fit the audio classifier
    print("🏋️ Training MARVIS on audio data...")
    
    try:
        audio_classifier.fit(
            audio_train_paths,
            y_train_audio,
            audio_test_paths[:10],  # Use subset for demo
            class_names=class_names_audio,
            task_type='classification'  # Explicitly specify task type
        )
        
        print("✅ Audio classifier trained successfully!")
        
        # Make predictions
        print("\n🔮 Making predictions on audio test data...")
        
        # Create temp directory for audio outputs
        temp_dir_audio = tempfile.mkdtemp(prefix='marvis_audio_demo_')
        
        # Evaluate on a small subset
        audio_results = audio_classifier.predict_with_detailed_results(
            audio_test_paths[:5],  # First 5 test samples
            y_test_audio[:5],
            save_visualizations=True,
            output_dir=temp_dir_audio
        )
        
        print("\n📈 Audio Classification Results:")
        accuracy = audio_results.get('accuracy', 'N/A')
        completed_samples = audio_results.get('completed_samples', 5)
        
        print(f"   • Accuracy: {accuracy}")
        print(f"   • Completed samples: {completed_samples}")
        
        # Display predictions
        predictions = audio_results.get('predictions', [])
        if predictions:
            print("\n🎯 Sample Audio Predictions:")
            for i, pred in enumerate(predictions[:3]):
                predicted_class = pred if isinstance(pred, str) else class_names_audio[pred] if pred < len(class_names_audio) else f"Class {pred}"
                true_class = class_names_audio[y_test_audio[i]]
                print(f"   Sample {i+1}: Predicted='{predicted_class}', True='{true_class}'")
        
        # Display audio visualization
        print("\n🖼️ Displaying t-SNE visualization for audio data...")
        
        viz_files_audio = list(Path(temp_dir_audio).glob("*.png"))
        
        if viz_files_audio:
            latest_viz_audio = max(viz_files_audio, key=lambda p: p.stat().st_mtime)
            print(f"📊 Found audio visualization: {latest_viz_audio.name}")
            
            display(IPImage(filename=str(latest_viz_audio), width=600))
            
            print("\n🎨 Audio Visualization Features:")
            print("   • Each point represents an audio sample in t-SNE space")
            print("   • Whisper embeddings capture audio frequency patterns")
            print("   • Different tones cluster based on frequency content")
            print("   • VLM recognizes patterns to classify audio by frequency")
        else:
            print("⚠️ No audio visualization found.")
            
    except Exception as e:
        print(f"❌ Error during audio training/evaluation: {e}")
        print("Audio classification requires Whisper and additional dependencies.")
else:
    print("⚠️ Skipping audio classification demo due to missing dependencies.")
    print("Audio classification would show:")
    print("   • Whisper embeddings of audio frequency patterns")
    print("   • t-SNE clustering of similar audio characteristics")
    print("   • VLM-based reasoning about audio frequency content")

## 🎯 Part 4: Multi-Visualization Reasoning

MARVIS also supports **multi-visualization** mode, where multiple dimensionality reduction techniques are combined for richer visual reasoning.

In [ ]:
# Demonstrate multi-visualization with tabular data
print("🔬 Demonstrating multi-visualization reasoning...")

# Initialize multi-viz classifier
multi_viz_classifier = MarvisTsneClassifier(
    modality="tabular",
    vlm_model_id="Qwen/Qwen2.5-VL-3B-Instruct",
    enable_multi_viz=True,  # Enable multi-visualization
    visualization_methods=['pca', 'tsne'],  # Use PCA and t-SNE
    layout_strategy='adaptive_grid',
    reasoning_focus='comparison',
    tsne_perplexity=15,
    tsne_max_iter=500,
    seed=42
)

print("✅ Multi-visualization classifier initialized!")
print("   • Methods: PCA (linear) + t-SNE (non-linear)")
print("   • Layout: Adaptive grid for comparison")
print("   • Focus: Cross-method pattern comparison")

In [ ]:
# Fit and evaluate multi-viz classifier
print("🏋️ Training multi-visualization classifier...")

try:
    multi_viz_classifier.fit(
        X_train_tab,
        y_train_tab,
        X_test_tab[:15],  # Use subset
        class_names=class_names_tab,
        task_type='classification'  # Explicitly specify task type
    )
    
    print("✅ Multi-viz classifier trained!")
    
    # Create temp directory for multi-viz outputs
    temp_dir_multi = tempfile.mkdtemp(prefix='marvis_multi_viz_')
    
    # Evaluate with multi-visualization
    print("\n🔮 Making multi-visualization predictions...")
    
    multi_viz_results = multi_viz_classifier.predict_with_detailed_results(
        X_test_tab[:8],
        y_test_tab[:8],
        save_visualizations=True,
        output_dir=temp_dir_multi
    )
    
    print("\n📈 Multi-Visualization Results:")
    accuracy = multi_viz_results.get('accuracy', 'N/A')
    completed_samples = multi_viz_results.get('completed_samples', 8)
    
    print(f"   • Accuracy: {accuracy}")
    print(f"   • Completed samples: {completed_samples}")
    
    # Display multi-visualization
    viz_files_multi = list(Path(temp_dir_multi).glob("*multi*.png"))
    
    if viz_files_multi:
        latest_viz_multi = max(viz_files_multi, key=lambda p: p.stat().st_mtime)
        print(f"\n📊 Found multi-visualization: {latest_viz_multi.name}")
        
        display(IPImage(filename=str(latest_viz_multi), width=800))
        
        print("\n🎨 Multi-Visualization Features:")
        print("   • Left panel: PCA projection (linear relationships)")
        print("   • Right panel: t-SNE projection (non-linear clusters)")
        print("   • VLM compares both views for robust classification")
        print("   • Cross-method consensus improves prediction confidence")
    else:
        print("⚠️ No multi-visualization found, but classifier is functional.")
        
except Exception as e:
    print(f"❌ Error in multi-visualization demo: {e}")
    print("Multi-visualization requires additional computational resources.")

## 📊 Part 5: Comparison and Summary

Let's compare the performance across different modalities and summarize key insights.

In [ ]:
# Summary of results across modalities
print("📊 MARVIS Performance Summary Across Modalities")
print("=" * 60)

results_summary = {
    "Tabular Data": {
        "samples": len(X_test_tab),
        "features": X_tabular.shape[1],
        "classes": len(class_names_tab),
        "accuracy": tabular_results.get('accuracy', 'N/A') if 'tabular_results' in locals() else 'N/A',
        "method": "Direct tabular embedding → t-SNE"
    },
    "Image Data": {
        "samples": len(X_test_img),
        "features": "8x8 pixels (64 features)",
        "classes": len(class_names_img),
        "accuracy": image_results.get('accuracy', 'N/A') if 'image_results' in locals() else 'N/A',
        "method": "Pixel features → t-SNE"
    }
}

if audio_available and 'audio_results' in locals():
    results_summary["Audio Data"] = {
        "samples": len(audio_test_paths),
        "features": "Whisper embeddings",
        "classes": len(class_names_audio),
        "accuracy": audio_results.get('accuracy', 'N/A'),
        "method": "Audio → Whisper → t-SNE"
    }

for modality, info in results_summary.items():
    print(f"\n🎯 {modality}:")
    print(f"   • Test samples: {info['samples']}")
    print(f"   • Features: {info['features']}")
    print(f"   • Classes: {info['classes']}")
    print(f"   • Accuracy: {info['accuracy']}")
    print(f"   • Method: {info['method']}")

print("\n🎉 Key MARVIS Advantages:")
print("   ✅ Unified interface across modalities (tabular, image, audio)")
print("   ✅ Visual reasoning using state-of-the-art VLMs")
print("   ✅ Interpretable predictions through visualization")
print("   ✅ Multi-visualization support for robust classification")
print("   ✅ Handles few-shot learning scenarios effectively")
print("   ✅ No need for extensive hyperparameter tuning")

In [ ]:
# Visualization comparison
print("🖼️ Visualization Techniques in MARVIS")
print("=" * 50)

viz_techniques = {
    "t-SNE": "Non-linear dimensionality reduction preserving local structure",
    "PCA": "Linear dimensionality reduction showing global variance patterns",
    "Multi-viz": "Combines multiple methods for comprehensive pattern analysis",
    "KNN connections": "Shows nearest neighbor relationships for context",
    "3D mode": "Three-dimensional projections for complex data structures"
}

for technique, description in viz_techniques.items():
    print(f"\n🔍 {technique}:")
    print(f"   {description}")

print("\n🧠 VLM Reasoning Process:")
reasoning_steps = [
    "1. Data is embedded into high-dimensional space",
    "2. Dimensionality reduction creates 2D/3D visualization", 
    "3. Visualization is rendered as an image with color coding",
    "4. VLM analyzes visual patterns, clusters, and relationships",
    "5. VLM provides structured prediction with reasoning",
    "6. Confidence scores based on pattern clarity and consistency"
]

for step in reasoning_steps:
    print(f"   {step}")

## 🚀 Next Steps and Advanced Usage

Congratulations! You've successfully explored MARVIS's capabilities across multiple data modalities. Here are some advanced features and next steps to explore:

### 🔧 Advanced Configuration Options

```python
# Advanced MARVIS configuration
advanced_classifier = MarvisTsneClassifier(
    modality="tabular",
    vlm_model_id="gpt-4o",  # Use more powerful VLM
    
    # Multi-visualization settings
    enable_multi_viz=True,
    visualization_methods=['pca', 'tsne', 'umap', 'spectral'],
    layout_strategy='hierarchical',
    reasoning_focus='consensus',
    
    # 3D visualization
    use_3d=True,
    
    # Enhanced KNN analysis
    use_knn_connections=True,
    nn_k=10,
    
    # High-quality rendering
    max_vlm_image_size=1200,
    image_dpi=150,
    
    # Semantic class names
    use_semantic_names=True
)
```

### 📚 Useful Resources

- **Examples directory**: Real-world datasets and evaluation scripts
- **Scripts directory**: Utility tools for metadata, visualization, and results management
- **Tests directory**: Comprehensive test suite and integration examples

### 🎯 Real-World Applications

- **Tabular**: Financial data, medical records, survey responses
- **Images**: Medical imaging, satellite imagery, document classification
- **Audio**: Speech recognition, music classification, environmental sound analysis

### 🛠️ Customization Options

- **Custom embeddings**: Bring your own embedding models
- **Domain-specific VLMs**: Use specialized vision-language models
- **Visualization layouts**: Design custom multi-viz arrangements
- **Prompt engineering**: Customize VLM reasoning prompts

In [ ]:
# Cleanup temporary directories
print("🧹 Cleaning up temporary files...")

temp_dirs = []
if 'temp_dir' in locals():
    temp_dirs.append(temp_dir)
if 'temp_dir_img' in locals():
    temp_dirs.append(temp_dir_img)
if 'audio_temp_dir' in locals():
    temp_dirs.append(audio_temp_dir)
if 'temp_dir_audio' in locals():
    temp_dirs.append(temp_dir_audio)
if 'temp_dir_multi' in locals():
    temp_dirs.append(temp_dir_multi)

for temp_dir_path in temp_dirs:
    try:
        shutil.rmtree(temp_dir_path)
        print(f"   ✅ Cleaned: {temp_dir_path}")
    except Exception as e:
        print(f"   ⚠️ Could not clean {temp_dir_path}: {e}")

print("\n🎉 MARVIS Getting Started tutorial completed!")
print("\n📖 Thank you for exploring MARVIS! Ready to classify your own data? 🚀")